In [1]:
pip install ultralytics opencv-python Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.6/914.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 14.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

# **import necessary libraries**

In [3]:
import tkinter as tk
import cv2
from PIL import Image, ImageTk
from ultralytics import YOLO

#Tkinter is Python’s standard library for creating graphical user interfaces (GUIs)
#OpenCV (commonly imported as cv2) is an open-source computer vision and image processing library. It handles operations like reading images or video frames from a webcam, image manipulation, drawing bounding boxes
#PIL (Python Imaging Library) — now maintained as Pillow — is a library for opening, manipulating, and saving many different image file formats.
# Ultralytics is the organization behind YOLOv8, a popular state-of-the-art object detection (and segmentation/classification) model

# **Configuration of the Application**

In [5]:
MODEL_PATH = "yolov8n.pt"
"""
- "yolov8n.pt" uses the official pretrained nano model (COCO dataset).
"""

CAMERA_INDEX = 0
"""
Index for the webcam or camera.
"""

CONF_THRESHOLD = 0.5
"""
Confidence threshold for displaying detection results.
Objects with confidence below this value won't be shown.
"""

"\nConfidence threshold for displaying detection results.\nObjects with confidence below this value won't be shown.\n"

# **Define Application Class**

In [6]:
# APPLICATION CLASS

class YOLOv8TkinterApp:
    """
   This class demonstrates real-time object recognition with YOLOv8 and displays the results in a Tkinter window.
    """

    def __init__(self, window, window_title, model_path=MODEL_PATH, camera_index=CAMERA_INDEX):
        """
        Initialize the Tkinter application.

        :param window: The root Tkinter window.
        :param window_title: Title of the Tkinter window.
        :param model_path: Path to the YOLOv8 model file (e.g., 'yolov8n.pt' or 'best.pt').
        :param camera_index: Index of the webcam or video source.
        """
        self.window = window
        self.window.title(window_title)

        #Step-1: Load YOLOv8 model
        self.model = YOLO(model_path)

        #Step-2. Set up video collecting
        self.cap = cv2.VideoCapture(camera_index)
        if not self.cap.isOpened():
            raise IOError(f"Cannot open camera (index={camera_index}).")

        #Step-3. Create a label widget to show the frames.
        self.display_label = tk.Label(self.window)
        self.display_label.pack()

        #Step-4. Start the video loop
        self.update_frame()

        #Step-5. Handle closing
        self.window.protocol("WM_DELETE_WINDOW", self.on_close)

    def update_frame(self):
        """
        Continuously read camera frames, perform YOLO detection, draw bounding boxes, and update Tkinter label with annotated images.
        """
        ret, frame = self.cap.read()
        if not ret:
            # If no frame is captured, stop trying
            return

        # Perform YOLOv8 inference on the frame.
        results = self.model.predict(frame, conf=CONF_THRESHOLD)

        # Extract the detection results (assuming one image/frame => results[0])
        boxes = results[0].boxes  # bounding boxes from YOLO
        for box in boxes:
            # Each 'box' has .cls (class), .conf (confidence), and .xyxy (coords)
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            # Convert float coords to int for drawing
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Get the class name using the model's 'names' dict
            class_name = self.model.names.get(cls_id, "Unknown")

            # Draw the bounding box on the frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
            # Create label text: "<class> <confidence>"
            label_text = f"{class_name} {conf:.2f}"
            # Place the label slightly above the bounding box
            cv2.putText(
                frame, label_text, (x1, max(y1 - 5, 0)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2
            )

        # Convert the OpenCV BGR image to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Convert to PIL Image
        img_pil = Image.fromarray(frame_rgb)
        # Convert to ImageTk for Tkinter
        imgtk = ImageTk.PhotoImage(image=img_pil)

        # Update label display
        self.display_label.imgtk = imgtk  # keep a reference
        self.display_label.configure(image=imgtk)

        # Schedule the next update in 10 ms (about 100 FPS max)
        self.window.after(10, self.update_frame)

    def on_close(self):
        """
        Cleanup resources when the Tkinter window is closed.
        """
        self.cap.release()
        self.window.destroy()


In [ ]:
if __name__ == "__main__":

    root = tk.Tk()
    app = YOLOv8TkinterApp(root, "YOLOv8 Real-Time Detection (Tkinter)")
    root.mainloop()